In [11]:
import pandas as pd
import numpy as np

In [12]:
train=pd.read_csv('C:/Users/82102/OneDrive - dongguk.edu/바탕 화면/2023.1학기 비어플/data/train.csv',encoding='utf-8')
test=pd.read_csv('C:/Users/82102/OneDrive - dongguk.edu/바탕 화면/2023.1학기 비어플/data/test.csv',encoding='utf-8')

## ID 열은 학습에 사용하지 않으므로 삭제

In [13]:
train.drop(['ID'], axis=1,inplace=True)
test.drop(['ID'], axis=1,inplace=True)

## target 인 STATUS 승인은 0, 승인거부는 1으로 변환

In [14]:
train['STATUS'] = train['STATUS'].replace({'승인':0, '승인거부':1})
test['STATUS'] = test['STATUS'].replace({'승인':0, '승인거부':1})

## 범주형 변수는 One-hot Encoding, 수치형 변수는 Scaling

- train, test  각각 따로 진행하여 set 만듦

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [16]:
cate_train = train[['CODE_GENDER','CNT_CHILDREN','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']]
conti_train = train[['AMT_INCOME_TOTAL','age','고용개월','MONTHS_BALANCE','begin','승인비율']]

cate_test = test[['CODE_GENDER','CNT_CHILDREN','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']]
conti_test = test[['AMT_INCOME_TOTAL','age','고용개월','MONTHS_BALANCE','begin','승인비율']]

y_train=train[['STATUS']]
y_test=test[['STATUS']]

In [17]:
cate_train_dummy=pd.get_dummies(cate_train)
cate_test_dummy=pd.get_dummies(cate_test)

scaler_train=StandardScaler()
conti_train_st=scaler_train.fit_transform(conti_train)
scaler_test=StandardScaler()
conti_test_st=scaler_test.fit_transform(conti_test)

In [18]:
conti_train_st=pd.DataFrame(conti_train_st, columns=conti_train.columns)
conti_test_st=pd.DataFrame(conti_test_st, columns=conti_test.columns)

cate_train_dummy.reset_index(inplace=True, drop=True)
conti_train_st.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
cate_test_dummy.reset_index(inplace=True, drop=True)
conti_test_st.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [19]:
train_set=pd.concat([conti_train_st,cate_train_dummy, y_train], axis=1)
test_set=pd.concat([conti_test_st,cate_test_dummy, y_test], axis=1)

In [20]:
train_set.to_csv('train_set.csv', index=False, encoding='UTF-8-sig')
test_set.to_csv('test_set.csv', index=False, encoding='UTF-8-sig')

## Data Sampling

In [10]:
train_maj=train_set[train_set['STATUS']==0] #130224 행
train_1=train_set[train_set['STATUS']==1] #2173행

In [11]:
train_2 = train_maj.sample(2173, random_state=2023)
train_34 = train_maj.drop(train_2.index)
train_3 = train_34.sample(round(len(train_34)/2), random_state=2023)
train_4 = train_34.drop(train_3.index)

print(f'train_1 : {len(train_1)}\ntrain_2 : {len(train_2)}\ntrain_3 : {len(train_3)}\ntrain_4 : {len(train_4)}\n')

train_1 : 2173
train_2 : 2173
train_3 : 64026
train_4 : 64025



In [12]:
train_3.loc[train_3['STATUS'].sample(round(len(train_3)/2), random_state = 2023).index, 'STATUS'] = 1

In [13]:
X_train=train_3.drop(['STATUS'], axis=1)
y_train=train_3['STATUS']

X_test_2 = train_2.drop(['STATUS'], axis=1)
y_test_2 = train_2['STATUS']
X_test_4 = train_4.drop(['STATUS'], axis=1)
y_test_4 = train_4['STATUS']

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

### 1. Logistic Regression

In [15]:
model = LogisticRegression(penalty = 'none', solver='saga',random_state=0) 
model.fit(X_train, y_train) # 모델 학습

y_pred_2 = model.predict(X_test_2) # 예측 라벨
print(f'Logistc train 2 Accuracy : {accuracy_score(y_pred_2, y_test_2)}')

y_pred_4 = model.predict(X_test_4) # 예측 라벨
print(f'Logistc train 4 Accuracy : {accuracy_score(y_pred_4, y_test_4)}')

C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Logistc train 2 Accuracy : 0.5167970547630004
Logistc train 4 Accuracy : 0.5136587270597422


### 2. Random Forest

In [16]:
model_log = RandomForestClassifier(random_state=0)
model_log.fit(X_train, y_train) # 모델 학습

y_pred_2 = model_log.predict(X_test_2) # 예측 라벨
print(f'Random Forest train 2 Accuracy : {accuracy_score(y_pred_2, y_test_2)}')

y_pred_4 = model_log.predict(X_test_4) # 예측 라벨
print(f'Random Forest train 4 Accuracy : {accuracy_score(y_pred_4, y_test_4)}')

Random Forest train 2 Accuracy : 0.500230096640589
Random Forest train 4 Accuracy : 0.5015853182350645


### 3.SVM

In [17]:
model_log = SVC(kernel = 'rbf',random_state=0)
model_log.fit(X_train, y_train) # 모델 학습

y_pred_2 = model_log.predict(X_test_2) # 예측 라벨
print(f'SVM train 2 Accuracy : {accuracy_score(y_pred_2, y_test_2)}')

y_pred_4 = model_log.predict(X_test_4) # 예측 라벨
print(f'SVM train 4 Accuracy : {accuracy_score(y_pred_4, y_test_4)}')

SVM train 2 Accuracy : 0.49056603773584906
SVM train 4 Accuracy : 0.4986333463490824


### 4. XGB

In [18]:
model_log = XGBClassifier(random_state=0)
model_log.fit(X_train, y_train) # 모델 학습

y_pred_2 = model_log.predict(X_test_2) # 예측 라벨
print(f'XGB train 2 Accuracy : {accuracy_score(y_pred_2, y_test_2)}')

y_pred_4 = model_log.predict(X_test_4) # 예측 라벨
print(f'XGB train 4 Accuracy : {accuracy_score(y_pred_4, y_test_4)}')

C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:34:40] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB train 2 Accuracy : 0.48182236539346524
XGB train 4 Accuracy : 0.4942912924638813


### 5. KNN

In [19]:
model_log = KNeighborsClassifier()
model_log.fit(X_train, y_train) # 모델 학습

y_pred_2 = model_log.predict(X_test_2) # 예측 라벨
print(f'KNN train 2 Accuracy : {accuracy_score(y_pred_2, y_test_2)}')

y_pred_4 = model_log.predict(X_test_4) # 예측 라벨
print(f'KNN train 4 Accuracy : {accuracy_score(y_pred_4, y_test_4)}')

KNN train 2 Accuracy : 0.49194661757938335
KNN train 4 Accuracy : 0.5011479890667708


In [20]:
train_1.to_csv('train_1.csv', index=False, encoding='UTF-8-sig')
train_2.to_csv('train_2.csv', index=False, encoding='UTF-8-sig')